In [8]:
from model import *
from data import *
from train_add import *
import matplotlib.pyplot as plt
import numpy as np

There is 2 folders of data:
- folder with images
- folder with mask wich have names same as correspondand image in imge folder

Function creat_train_data_frame will cread dataframe which contain file path for mask and file path for image for every training example

In [ ]:
df = creat_train_data_frame()

data_gen_args = dict(rotation_range=0,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=0.1,
                    zoom_range=0.01,
                    horizontal_flip=True,
                    fill_mode='nearest')

myGene = trainGenerator(1,df,data_gen_args,save_to_dir = None)

In [ ]:
print(df)

Next cell would visualise specific number of outputs from training data generator

In [ ]:
for k in range(0,30):
    im,mask = next(myGene)
    plt.imshow(im[0,:,:,0])
    plt.colorbar()
    plt.show()
    
    plt.imshow(mask[0,:,:,0])
    plt.colorbar()
    plt.show()
    
    plt.imshow(m2[0,:,:,0])
    plt.colorbar()
    plt.show()
    disp(im,mask)

Checking shcedule of learning rate and creating schedule callback

In [2]:
e = np.linspace(0,1000,1000)
l = []
for ee in e:
    l.append(scheduler(ee))
plt.plot(e,l)

In [ ]:
callback_sch = tf.keras.callbacks.LearningRateScheduler(scheduler)

As we deal with crack, which gradualy grow there is both very short crack with lenght on image about 10 pixels and large crack wich take all high of image. Mask contain too classes:
- 0 - There is NO crack on correspondent pixel
- 1 - There is crack on correspondent pixel

Area of any crack(short or long) is small reletive to specimen zone, unefected by crack, so the quantity of 0 pixels is much more than 1 in training set. To make training more robust, the loss weighting should be implimented so that 0 pixels would be not more preferable for model due to its numerous representatives in compare to 1 class. If take loss weights according to hole data set and model would there would be single weightes (in case of presented data set it is 1500). That means that in case when model would misslabel 1 class it would be punished in weight time more than punished in case of misslabeling 0 class. But in such case, short crack would beadly detectable becouse it have less ratio of 1 class to 0 class than long crack




weighted_cross_entropy function calculate function and weightes 1 class not according to whole data set but with respect to single current mask 1 to 0 ratio. That mean that for short crack, 1 class loss weights would be biger than for long crack.

In [ ]:
model = unet()
model.compile(optimizer='adam',loss = weighted_cross_entropy(1))

Creat callback for saving weights of model to specified folder

In [ ]:
filepath = '/content/drive/My Drive/ML/Skynet keypoint/checkpoint1/'
save_callback=tf.keras.callbacks.ModelCheckpoint(filepath, monitor='train_loss', verbose=0,
                                                 save_best_only=False,save_weights_only=True, mode='auto', save_freq=1000)

Training

In [ ]:
hist = model.fit_generator(myGene,steps_per_epoch=50,epochs=500,callbacks=[save_callback,callback_sch])

In [ ]:
plt.plot(hist.history['loss'])

Checking trained model on validation data

In [ ]:
val_read = val_name_list()
k=0
while 1:

    im = plt.imread(val_read[k])
    k=k+1
    im = np.expand_dims(im,axis=0)
    w = im.shape[2]
    w1 = int(w*0.3)
    w2 = int(w*0.7)
    im = im[:,:,w1:w2,:]

    print(im.shape)




    m=model.predict(im)
    m2 = m.copy()
    th = 0.5
    m2[m>=th] = 1
    m2[m<th] = 0




    plt.imshow(m[0,:,:,0])
    plt.colorbar()
    plt.show()
    plt.imshow(m2[0,:,:,0])
    plt.colorbar()
    plt.show()
    plt.imshow(im[0,:,:,:])
    plt.colorbar()
    plt.show()
    disp2(im,m2)